In [ ]:
# Snowflake python API
from snowflake.core import Root

# Snowpark
from snowflake.snowpark.context import get_active_session
session = get_active_session()
env = 'DEV'
session.use_schema('BUILD_PYTHON_DE.DATA')

In [ ]:
from snowflake.core.task.dagv1 import DAGOperation, DAG, DAGTask
from datetime import timedelta

# DAG API 를 사용해서 task 생성
warehouse_name = "DEMO_WH"
dag_name = "DEMO_DAG"

api_root = Root(session)
schema = api_root.databases['BUILD_PYTHON_DE'].schemas['DATA']
dag_op = DAGOperation(schema)

# DAG 정의
with DAG(dag_name, schedule=timedelta(days=1), warehouse=warehouse_name) as dag:
    dag_task1 = DAGTask("stats_customer", definition=f'''EXECUTE NOTEBOOK BUILD_PYTHON_DE.DATA."{env}_STATS_CUSTOMER"()''', warehouse=warehouse_name)
    dag_task2 = DAGTask("stats_sales", definition=f'''EXECUTE NOTEBOOK BUILD_PYTHON_DE.DATA."{env}_STATS_SALES"()''', warehouse=warehouse_name)

    # tasks 간 dependencies 정의
    dag_task1 >> dag_task2 # dag_task1 실행 후 dag_task2 실행

# Snowflake 에 DAG 생성
dag_op.deploy(dag, mode="orreplace")

In [ ]:
dagiter = dag_op.iter_dags(like='demo_dag%')
for dag_name in dagiter:
    print(dag_name)

dag_op.run(dag)